# 🔗 Feature Merge & Test Set Application

이 노트는 앞서 생성한 **'선수 성향(Player Persona)'** 정보를 학습 데이터와 **테스트 데이터**에 적용하는 단계입니다.
테스트 데이터에도 똑같이 적용해주어야 모델이 실전에서 "아, 이 선수는 원래 이렇게 차지!"라고 판단할 수 있습니다.

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
from tqdm import tqdm
import gc

warnings.filterwarnings('ignore')

# 1. Load Data
PREPROCESS_DIR = './data_preprocess'
OPEN_DIR = './open_track1'

# 학습 데이터 (기존 전처리된 것 or 원본)
# 여기서는 원본에 붙이는 과정을 보여줌 (또는 enriched가 있다면 그것 활용)
train = pd.read_csv(os.path.join(OPEN_DIR, 'train.csv'))

# 생성한 피처 로드
player_stats = pd.read_csv(os.path.join(PREPROCESS_DIR, 'player_features.csv'))
game_states = pd.read_csv(os.path.join(PREPROCESS_DIR, 'train_game_states.csv'))

print("Data Loaded")

## 2. Merge into Train
학습 데이터에 피처를 붙입니다.
- `player_stats`: `player_id` 기준 병합
- `game_states`: `game_id`, `time_seconds` ... 기준 병합 (여기서는 간단히 index가 맞는다면 concat도 가능하지만, key merge가 안전)

In [ ]:
# Player Stats Merge
train_merged = pd.merge(train, player_stats, on='player_id', how='left')

# 결측치 처리 (정보가 없는 선수는 평균값으로 대체)
fill_values = {
    'avg_pass_dist': player_stats['avg_pass_dist'].mean(),
    'avg_pass_angle': player_stats['avg_pass_angle'].mean(),
    'pass_count': 0
}
train_merged.fillna(fill_values, inplace=True)

# Game States Merge (옵션: 테스트셋에 점수를 못 구하면 제외해야 함)
# 일단 Train에는 붙여둠 (분석용)
train_merged = pd.merge(train_merged, game_states[['game_id', 'period_id', 'time_seconds', 'score_diff']], 
                        on=['game_id', 'period_id', 'time_seconds'], how='left')

print(f"Train Shape: {train_merged.shape}")
train_merged.head()

## 3. Process Test Set
엄청 중요한 단계입니다. 테스트 데이터는 수많은 작은 파일들로 쪼개져 있습니다.
`test.csv`를 보고 하나씩 읽어서 피처를 붙이고, 하나의 큰 파일(또는 메모리 객체)로 합쳐야 합니다.

In [ ]:
test_index = pd.read_csv(os.path.join(OPEN_DIR, 'test.csv'))

# 테스트 데이터 통합 리스트
test_list = []

print("Processing Test Files...")
for idx, row in tqdm(test_index.iterrows(), total=len(test_index)):
    file_path = os.path.join(OPEN_DIR, row['path'])
    df_temp = pd.read_csv(file_path)
    
    # Player Stats 붙이기
    df_temp = pd.merge(df_temp, player_stats, on='player_id', how='left')
    
    # 결측치 채우기 (Train 평균으로)
    df_temp.fillna(fill_values, inplace=True)
    
    # Score Diff 채우기
    # 주의: 테스트셋에서는 실시간 점수를 알 수 없음 (0으로 가정하거나 NaN)
    # 여기서는 안전하게 0으로 처리 (모델이 점수차를 모르면 평소대로 차게 둠)
    df_temp['score_diff'] = 0 
    
    # 마지막 행만 예측 대상이지만, 시퀀스 모델을 위해 전체를 저장하거나,
    # 트리 모델을 위해 마지막 행만 가져오거나 해야 함.
    # 여기서는 **마지막 행(예측 대상)**만 추출해서 저장 (전통적인 방식)
    # 만약 시퀀스(딥러닝)를 쓴다면 전체를 저장해야 함.
    
    # 일단은 '트리 모델'용으로 마지막 행만 추출하여 병합
    last_row = df_temp.iloc[[-1]].copy()
    test_list.append(last_row)

test_merged = pd.concat(test_list, ignore_index=True)
print(f"Test Merged Shape: {test_merged.shape}")

## 4. Save Final Datasets
이제 모델에 바로 넣을 수 있는 `train_final.csv`와 `test_final.csv`가 완성되었습니다.

In [ ]:
# 저장
# 예측을 위한 마지막 전처리들 (One-hot encoding 등)은 모델링 단계에서 하거나 여기서 할 수 있음.
 # 여기서는 Feature 추가까지만 하고 저장.

train_merged.to_csv(os.path.join(PREPROCESS_DIR, 'train_final.csv'), index=False)
test_merged.to_csv(os.path.join(PREPROCESS_DIR, 'test_final.csv'), index=False)

print("Files Saved:")
print(f" - {os.path.join(PREPROCESS_DIR, 'train_final.csv')}")
print(f" - {os.path.join(PREPROCESS_DIR, 'test_final.csv')}")